# BlueMath-tk Model Wrapper Tutorial: Parametric Study Setup

This notebook demonstrates how to use BlueMath-tk's Model Wrapper system to set up and execute parametric studies with numerical models. We'll use a `DummyModelWrapper` as an example, but the same principles apply to real model wrappers like SWASH, SWAN, Delft3D, etc.

## 1. Import Required Libraries

We start by importing the necessary modules for our parametric study setup.

In [4]:
from bluemath_tk.wrappers._base_wrappers import DummyModelWrapper
import pandas as pd
import itertools

## 2. Define Parameter Space

The first step is defining the parameter space for our study. Each parameter can have multiple values that we want to test in different combinations.

In [5]:
fields = {
    "manning": [1, 2, 3],      # Manning roughness coefficient values
    "alpha": [4, 5, 6],        # Alpha parameter values  
    "dy": [0.1, 0.2]           # Grid spacing values
}

print("Parameter space defined:")
for param, values in fields.items():
    print(f"  {param}: {values}")

Parameter space defined:
  manning: [1, 2, 3]
  alpha: [4, 5, 6]
  dy: [0.1, 0.2]


We use `itertools.product()` to create all possible combinations of parameter values. This creates a Cartesian product - every combination of parameters will be tested.

In [6]:
# Get parameter names and create all combinations
keys = fields.keys()
combinations = list(itertools.product(*fields.values()))

print(f"Total number of combinations: {len(combinations)}")
print("First 3 combinations:")
for i, combo in enumerate(combinations[:3]):
    print(f"  Combination {i+1}: {dict(zip(keys, combo))}")

Total number of combinations: 18
First 3 combinations:
  Combination 1: {'manning': 1, 'alpha': 4, 'dy': 0.1}
  Combination 2: {'manning': 1, 'alpha': 4, 'dy': 0.2}
  Combination 3: {'manning': 1, 'alpha': 5, 'dy': 0.1}


We convert the combinations to a pandas DataFrame for easier manipulation and add any derived parameters (like `nmax` calculated from `dy`).

In [7]:
# Create DataFrame from combinations
df = pd.DataFrame(combinations, columns=keys)

# Add derived parameter
df["nmax"] = 0.5 * df["dy"]

print("Parameter combinations DataFrame:")
print(df.head())
print(f"\nDataFrame shape: {df.shape}")

Parameter combinations DataFrame:
   manning  alpha   dy  nmax
0        1      4  0.1  0.05
1        1      4  0.2  0.10
2        1      5  0.1  0.05
3        1      5  0.2  0.10
4        1      6  0.1  0.05

DataFrame shape: (18, 4)


The Model Wrapper expects parameters in dictionary format where each key maps to a list of values. We convert our DataFrame to this format.

In [8]:
# Convert DataFrame to dictionary format expected by model wrapper
metamodel_parameters = df.to_dict('list')

print("Metamodel parameters format:")
for param, values in metamodel_parameters.items():
    print(f"  {param}: {values[:3]}... (showing first 3 values)")

Metamodel parameters format:
  manning: [1, 1, 1]... (showing first 3 values)
  alpha: [4, 4, 5]... (showing first 3 values)
  dy: [0.1, 0.2, 0.1]... (showing first 3 values)
  nmax: [0.05, 0.1, 0.05]... (showing first 3 values)


## 3. Set Up Directory Structure and Fixed Parameters

Define where templates are stored, where outputs should go, and any parameters that remain constant across all model runs.

In [9]:
# Set root_dir to cwd for the model wrapper
import os
root_dir = os.getcwd()  # Use current working directory as root directory
templates_dir = os.path.join(root_dir,"DummyTemplates")          # Directory containing input file templates
output_dir = os.path.join(root_dir,"DummyOutput")                # Directory for model outputs
fixed_parameters = {"advection": 1}                         # Parameters that don't vary

print(f"Templates directory: {templates_dir}")
print(f"Output directory: {output_dir}")
print(f"Fixed parameters: {fixed_parameters}")

Templates directory: /vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyTemplates
Output directory: /vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput
Fixed parameters: {'advection': 1}


## 4. Initialize the Model Wrapper

Create an instance of the model wrapper with our configuration. The wrapper will handle case generation, template rendering, and model execution.

In [12]:
hurrywaves_model = DummyModelWrapper(
    templates_dir=templates_dir,
    metamodel_parameters=metamodel_parameters,
    fixed_parameters=fixed_parameters,
    output_dir=output_dir,
)

hurrywaves_model.build_cases(mode="one_by_one")

print(f"Number of case directories created: {len(hurrywaves_model.cases_dirs)}")
print("First 3 case directories:")
for i, case_dir in enumerate(hurrywaves_model.cases_dirs[:3]):
    print(f"  Case {i+1}: {case_dir}")

Number of case directories created: 18
First 3 case directories:
  Case 1: /vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput/0000
  Case 2: /vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput/0001
  Case 3: /vols/abedul/home/grupos/valvanuz/HySwash/BlueMath/toolkit/wrappers/DummyOutput/0002


## 9. Execute Model Runs in Bulk

The `run_cases_bulk()` method executes all cases using a single command. Here we're using SLURM's job array feature to run cases in parallel on a cluster.

In [ ]:
# Execute all cases using SLURM job arrays
hurrywaves_model.run_cases_bulk(
    launcher="sbatch --array=0-799 launcher.sh"
)

print("Bulk execution command submitted!")

## Understanding the Workflow

### What happens behind the scenes:

1. **Case Generation**: Each parameter combination becomes a separate case with its own directory
2. **Template Rendering**: Input file templates are filled with parameter values using Jinja2
3. **Bulk Execution**: All cases are submitted as a job array to the cluster scheduler

### Key Concepts:

- **Metamodel Parameters**: Variables that change between cases (our parameter space)
- **Fixed Parameters**: Constants that remain the same across all cases  
- **Templates**: Jinja2 template files that get rendered with parameter values
- **Bulk Execution**: Running all cases with a single scheduler command

### Alternative Execution Methods:

Instead of `run_cases_bulk()`, you could also use:
- `run_cases(launcher="serial")` - Run cases sequentially
- `run_cases(launcher="mpi", num_workers=4)` - Run cases in parallel locally

## Next Steps

After model execution completes, you would typically:
1. Use `postprocess_cases()` to extract results from model outputs
2. Analyze the relationship between input parameters and model results
3. Build metamodels or response surfaces from the data